<a href="https://colab.research.google.com/github/YogeshThakare007/World-Bank-Data-EDA/blob/main/World%20Bank%20EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exploratory Data Analysis on the World Bank **Data**
The objective of this project is to collect data from the World Bank Open APIs and prepare and analyse the data using Python.


In [1]:
import pandas as pd
import numpy as np
import requests

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Base URL used in all the API calls
BASE_URL='http://api.worldbank.org/v2/'

# List of indicators according to the features defined above
INDICATOR_CODES=['SP.POP.TOTL', 'SP.POP.TOTL.FE.IN', 'SP.POP.TOTL.MA.IN',
 'SP.DYN.CBRT.IN', 'SP.DYN.CDRT.IN',
 'SE.COM.DURS',
 'SL.IND.EMPL.ZS', 'SL.AGR.EMPL.ZS', 'SL.AGR.EMPL.FE.ZS', 'SL.IND.EMPL.FE.ZS', 'SL.UEM.TOTL.ZS',
 'NY.GDP.MKTP.CD',
 'NY.ADJ.NNTY.PC.KD.ZG', 'NY.GSR.NFCY.CD', 'NV.AGR.TOTL.CD',
 'EG.USE.ELEC.KH.PC', 'EG.FEC.RNEW.ZS', 'EG.USE.COMM.FO.ZS']

COUNTRY_LIST=['USA', 'India', 'China', 'Japan', 'Canada', 'Great Britain', 'South Africa']

# mapping of feature codes to more meaningful names
featureMap={
    "SP.POP.TOTL": "Total Population",
    "SP.POP.TOTL.FE.IN": "Female Population",
    "SP.POP.TOTL.MA.IN": "Male Population",
    "SP.DYN.CBRT.IN": "Birth Rate",
    "SP.DYN.CDRT.IN": "Death Rate",
    "SE.COM.DURS": "Compulsory Education Dur.",
    "SL.IND.EMPL.ZS":"Employment in Industry(%)",
    "SL.AGR.EMPL.ZS": "Employment in Agriculture(%)",
    "SL.AGR.EMPL.FE.ZS": "Female Employment in Agriculture(%)",
    "SL.IND.EMPL.FE.ZS": "Female Employment in Industry(%)",
    "SL.UEM.TOTL.ZS": "Unemployment(%)",
    "NY.GDP.MKTP.CD": "GDP in USD",
    "NY.ADJ.NNTY.PC.KD.ZG":"National Income per Capita",
    "NY.GSR.NFCY.CD":"Net income from Abroad",
    "NV.AGR.TOTL.CD":"Agriculture value added(in USD)",
    "EG.USE.ELEC.KH.PC":"Electric Power Consumption(kWH per capita)",
    "EG.FEC.RNEW.ZS":"Renewable Energy Consumption (%)",
    "EG.USE.COMM.FO.ZS":"Fossil Fuel Consumption (%)"
}

# Mapping of country codes to their actual names
countryMap={
    "US": "USA",
    "IN":"India",
    "CN": "China",
    "JP": "Japan",
    "CA": "Canada",
    "GB": "Great Britain",
    "ZA": "South Africa"
}

# constant parameters used in sending the request.
params = dict()
# to ensure we receive a JSON response
params['format']='json'
# The data we fetch is for 59 years.
# Hence we change the default page size of 50 to 100 to ensure we need only one API call per feature.
params['per_page']='100'
# Range of years for which the data is needed
params['date']='1960:2018'

In [5]:
# Function to get JSON data from the endpoint
def loadJSONData(country_code): 
    dataList=[]
    
    # iterate over each indicator code specified in the contant INDICATOR_CODES defined above
    for indicator in INDICATOR_CODES: 
        
        # form the URL in the desired format
        # E.g: http://api.worldbank.org/v2/countries/us/indicators/SP.POP.TOTL?format=json&per_page=200&date=1960:2018
        url=BASE_URL+'countries/'+country_code.lower()+'/indicators/'+indicator
        
        # send the request using the resquests module
        response = requests.get(url, params=params)
        
        # validate the response status code
        # The API returns a status_code 200 even for error messages,
        # however, the response body contains a field called "message" that includes the details of the error
        # check if message is not present in the response
        if response.status_code == 200 and ("message" not in response.json()[0].keys()):
            # print("Successfully got data for: " + str(featureMap[indicator]))
            
            # list of values for one feature
            indicatorVals=[]
            
            # the response is an array containing two arrays - [[{page: 1, ...}], [{year: 2018, SP.POP.TOTL: 123455}, ...]]
            # hence we check if the length of the response is >1
            if len(response.json()) > 1:
                
                # if yes, iterate over each object in the response
                # each object gives one single value for each year
                for obj in response.json()[1]:
                    
                    # check for empty values
                    if obj['value'] is "" or obj['value'] is None:
                        indicatorVals.append(None)
                    else:
                    # if a value is present, add it to the list of indicator values
                        indicatorVals.append(float(obj['value']))
                dataList.append(indicatorVals)
        else:
            # print an error message if the API call failed
            print("Error in Loading the data. Status Code: " + str(response.status_code))
            
    # Once all the features have been obtained, add the values for the "Year"
    # The API returns the indicator values from the most recent year. Hence, we create a list of years in reverse order
    dataList.append([year for year in range(2018, 1959, -1)])
    # return the list of lists of feature values [[val1,val2,val3...], [val1,val2,val3...], [val1,val2,val3...], ...]
    return dataList

#----------------------------------------------------------------------------------------------------

# function to invokde the loadJSONData function and form the final DataFrame for each country
def getCountrywiseDF(country_code):
    
    # The resulting dataframe needs to have meaningful column names
    # hence we create a list of column names from the map defined above
    col_list=list(featureMap.values())
    # append the year column name
    col_list.append('Year')
    
    print("------------------Loading data for: "+countryMap[country_code]+"-----------------------")
    
    # for the given country call the loadJSONData function and fetch the data from the API
    dataList=loadJSONData(country_code)
    
    # transform the list of lists of features into a DataFrame
    # np.column_stack is used to add each list as a column 
    df=pd.DataFrame(np.column_stack(dataList), columns=col_list)
    
    # add the country column by extracting the country name from the map using the country code
    df['Country'] = countryMap[country_code]
    
    # display the resulting dataframe
    display(df.head())
    
    # return the formed dataframe for the given country
    return df
# Call the getCountrywiseDF function with the code of each country under consideration
# We will have a seperate dataframe for each country - 7 data frames

US_df=getCountrywiseDF('US')
IN_df=getCountrywiseDF('IN')
CN_df=getCountrywiseDF('CN')
JP_df=getCountrywiseDF('JP')
CA_df=getCountrywiseDF('CA')
GB_df=getCountrywiseDF('GB')
ZA_df=getCountrywiseDF('ZA')

print("Data Loading Completed")


<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-46ac65fd9de2>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if obj['value'] is "" or obj['value'] is None:


------------------Loading data for: USA-----------------------


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Female Employment in Agriculture(%),Female Employment in Industry(%),Unemployment(%),GDP in USD,National Income per Capita,Net income from Abroad,Agriculture value added(in USD),Electric Power Consumption(kWH per capita),Renewable Energy Consumption (%),Fossil Fuel Consumption (%),Year,Country
0,326838199.0,164926348.0,161911851.0,11.6,8.678,12.0,19.56929,1.708719,0.975559,8.580613,3.9,20533057312000.0,1.83305,290307000000.0,185598437000.0,None,10.12,None,2018,USA
1,325122128.0,164151818.0,160970309.0,11.8,8.638,12.0,19.42705,1.768335,0.961346,8.535977,4.36,19477336549000.0,2.135014,292949000000.0,184424799000.0,None,9.92,None,2017,USA
2,323071755.0,163224028.0,159847727.0,12.2,8.493,12.0,19.47486,1.781334,0.97919,8.532497,4.87,18695110842000.0,-0.111701,231944000000.0,176228444000.0,None,9.46,None,2016,USA
3,320738994.0,162158414.0,158580581.0,12.4,8.44,12.0,19.55526,1.781391,0.969498,8.661249,5.28,18206020741000.0,3.214351,220383000000.0,188238199000.0,None,9.03,82.427828,2015,USA
4,318386329.0,161084758.0,157301571.0,12.5,8.237,12.0,19.70807,1.661639,0.888076,8.714112,6.17,17550680174000.0,2.182302,235226000000.0,205705398000.0,12993.965579,9.22,83.089042,2014,USA


------------------Loading data for: India-----------------------


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Female Employment in Agriculture(%),Female Employment in Industry(%),Unemployment(%),GDP in USD,National Income per Capita,Net income from Abroad,Agriculture value added(in USD),Electric Power Consumption(kWH per capita),Renewable Energy Consumption (%),Fossil Fuel Consumption (%),Year,Country
0,1369003306.0,661854076.0,707149230.0,17.651,6.644,8.0,24.9491,43.32882,55.49442,17.34357,7.65,2702929718960.459961,4.695242,-28935365554.060299,433323710255.151978,None,32.82,None,2018,India
1,1354195680.0,654607791.0,699587889.0,17.911,6.593,8.0,24.84621,43.93962,56.25889,17.63662,7.733,2651472946375.049805,7.15274,-28673171869.012299,439039677566.794983,None,32.41,None,2017,India
2,1338636340.0,647012921.0,691623419.0,18.514,6.603,8.0,24.71281,44.52136,56.95825,17.88424,7.842,2294797980509.009766,6.265538,-46857856463.203697,375516164816.913025,None,33.02,None,2016,India
3,1322866505.0,639323292.0,683543213.0,18.765,6.67,8.0,24.58389,45.15931,57.70898,18.14038,7.915,2103587813812.75,7.580204,-24405488069.835098,340244981475.059998,None,33.4,None,2015,India
4,1307246509.0,631697152.0,675549357.0,19.049,6.79,8.0,24.52794,45.7835,58.40867,18.4407,7.981,2039127446299.300049,5.69479,-24112069637.705299,342408945451.364014,797.349232,33.85,73.576979,2014,India


...

------------------Loading data for: Great Britain-----------------------


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Female Employment in Agriculture(%),Female Employment in Industry(%),Unemployment(%),GDP in USD,National Income per Capita,Net income from Abroad,Agriculture value added(in USD),Electric Power Consumption(kWH per capita),Renewable Energy Consumption (%),Fossil Fuel Consumption (%),Year,Country
0,66460344.0,33662065.0,32798279.0,11.0,9.2,11.0,18.12584,1.070025,0.618907,7.469033,4.0,2878152147315.819824,0.927874,-39229883074.598198,16317101948.0858,None,10.99,None,2018,Great Britain
1,66058859.0,33477363.0,32581496.0,11.4,9.2,11.0,18.26126,1.163051,0.650689,7.691214,4.33,2683399006715.790039,2.018576,-28893082281.108501,15896986741.0357,None,9.7,None,2017,Great Britain
2,65611593.0,33270675.0,32340918.0,11.8,9.1,11.0,18.47305,1.125685,0.627984,7.626142,4.81,2699659680997.200195,0.990839,-63118855915.297699,15477270586.058399,None,8.6,None,2016,Great Britain
3,65116219.0,33041408.0,32074811.0,11.9,9.2,11.0,18.66107,1.138458,0.647011,7.69417,5.3,2934857946213.470215,2.432155,-63810692067.091301,18473888200.619301,None,8.61,80.351771,2015,Great Britain
4,64602298.0,32802913.0,31799385.0,12.0,8.8,11.0,18.96643,1.25617,0.713861,7.959467,6.11,3065223279583.790039,3.14538,-54795748844.857803,22840749218.205799,5130.390253,7.36,82.716578,2014,Great Britain


------------------Loading data for: South Africa-----------------------


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Female Employment in Agriculture(%),Female Employment in Industry(%),Unemployment(%),GDP in USD,National Income per Capita,Net income from Abroad,Agriculture value added(in USD),Electric Power Consumption(kWH per capita),Renewable Energy Consumption (%),Fossil Fuel Consumption (%),Year,Country
0,57339635.0,29517286.0,27822349.0,21.137,8.765,9.0,20.58729,15.69041,16.57739,10.02311,24.22,404159690890.846008,-0.146707,-11288811310.346901,9158617157.11408,None,10.19,None,2018,South Africa
1,56641209.0,29175940.0,27465269.0,20.643,8.802,9.0,20.53531,16.79855,17.66136,10.48616,23.99,381448814653.455994,2.347842,-10575832463.249901,9512565346.393351,None,10.45,None,2017,South Africa
2,56422274.0,29004108.0,27418165.0,20.25,8.924,9.0,20.73353,16.05315,16.33636,10.41636,24.02,323585509674.481018,-1.373576,-8291430980.09355,7811956311.86773,None,10.53,None,2016,South Africa
3,55876504.0,28721087.0,27155417.0,21.3,9.259,9.0,21.34431,15.46671,16.11247,10.32776,22.87,346709790458.562988,1.273791,-7943171747.43347,7745421796.45039,None,10.28,None,2015,South Africa
4,54729551.0,28228865.0,26500685.0,22.057,9.432,9.0,21.14231,14.02137,14.73459,10.71216,22.61,381198869776.106018,0.083527,-9453635055.88138,8103329441.92394,4183.82749,9.85,86.791432,2014,South Africa


Data Loading Completed
